<img align="left" style="padding-right:30px;" src="./tutorial/logo_UNSW.svg" height="100"><img align="left" style="padding-right:30px;" src="./tutorial/Green_RGB.png" height="90"><img align="left" style="padding-right:30px;" src="./tutorial/image004.png" height="100">



[![DOI](https://zenodo.org/badge/940091341.svg)](https://doi.org/10.5281/zenodo.14939868) <a href="https://www.globh2e.org.au/"><img src="https://img.shields.io/badge/ARC:Funding%20number-IC200100023-blue.svg"/></a>

# **<div style="text-align: left; font-size: 30px;"> Pathways to global hydrogen production within planetary boundaries**</div>
**<div style="text-align: left; font-size: 25px;"> Life cycle optimisation - Tutorial**</div>

<div style="text-align: left; font-size: 16px;">Michaël Lejeune<sup>a,b</sup>, Sami Kara<sup>a,b</sup>, Michael Zwicky Hauschild<sup>c,d</sup>, Sareh Sharabifarahni<sup>a</sup>, Rahman Daiyan<sup>b,e</sup></div><br>

<div style="text-align: left; font-size: 13px;"><sup>a</sup>Sustainability in Manufacturing and Life Cycle Engineering Research Group, School of Mechanical and Manufacturing Engineering, the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>b</sup>Australian Research Council Training Centre for the Global Hydrogen Economy (GlobH2e), the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>c</sup>Centre for Absolute Sustainability, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>d</sup>Section for Quantitative Sustainability Assessment (QSA), Department of Environmental and Resource Engineering, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>e</sup>School of Minerals and Energy Engineering, The University of New South Wales, Sydney 2052, Australia</div><br>


> <span style="color:rgba(22, 210, 69, 1); font-weight: bold;">Target audience</span><br>
>- Life cycle assessment practioners/researchers
>- Absolute environmental sustainability practioners/researchers

> <span style="color:rgba(191, 200, 30, 1); font-weight: bold;">Prerequisites</span><br>
>- Background in life cycle assessment
>- Brightway
>- Activity browser (with ScenarioLink installed)
>- Julia with JuMP.jl
>- CPLEX optimisation solver
>- IDE (e.g., VS code)

# <span style="color:rgba(0, 0, 0, 1); font-weight: bold;">Contents</span><br>
>1. Background information on computational LCA and optimisation
>2. Installation of planetary boundaries characterisation factors
>3. Modelling aggregated inventories on Activity - browser
>4. pre-optimisation formating on Excel
>5. Implementation of the optimisaiton model in Julia using JuMP.jl



# **Case study**

<!-- > <span style="color:rgba(184, 193, 29, 1); font-weight: bold;">Simple optimisation example</span><br>
>- Optimisation of electrical mix for hydrogen production via PEM electrolysis
>- Using activity browser, excel and Julia (a python version is also possible)
>- Integration of prospective LCA data in the optimisation.
>- Optional - Cost and material tracking -->


> <span style="color:rgba(26, 190, 67, 1); font-weight: bold;">Goal  & Scope definition</span><br>
>- Goal: Minimising the effective planetary footprint of hydrogen production via PEM electrolysis.
>- LCA type: prospective attributional LCA (note that the appraoch can also be interpreted as consequential)
>- Functional unit: 225 MtH₂/year (2050) (global annual production of H₂ via water electrolysis)
>- Scope: Cradle to gate (electricity production + PEM electrolysis)
>- Database: ecoinvent 3.9.1 (cut-off) - updated with premise - SSP1-PkBudg500
<div style="text-align:center;">
  <img src="./tutorial/PEM_unit1.svg">
</div>


### **Initialisation**

In [71]:
using JuMP, CPLEX,Distributions,lce,DataFrames,LinearAlgebra
using PyPlot
using SparseArrays
using JLD2


In [72]:
initProject("Natcoms")


┌ Info: Project Natcoms -> model:ecoinvent_3.9.1 RCP:0.0 SSP:0 year:0 loaded
└ @ lce /Users/mickael/Library/CloudStorage/OneDrive-UNSW/Research/Publications/Journal articles/1_Natcoms/code/lce/src/lce.jl:781


Project              Model          RCP      SSP      Year   Method        
──────────────────── ────────────── ──────── ──────── ────── ──────────────
Natcoms              ecoinvent_3.9.1 0.0      0             0 Planetary boundaries

Natcoms: ecoinvent_3.9.1 0.0 0 0 Planetary boundaries


In [29]:
@load "../Source data/03_additional_data/1_00_total_human_impact/data_interaction_matrices.jld2" 𝚪ᵦ 𝚪ₕ catnames_ticks


3-element Vector{Symbol}:
 :𝚪ᵦ
 :𝚪ₕ
 :catnames_ticks

In [74]:
catnames_ticks


10-element Vector{String}:
 "Climate\nchange\nEnergy\nimbalance"
 "Climate\nchange CO2\nConcentration"
 "Ocean\nacidification"
 "Atmospheric\naerosol\nloading"
 "Freshwater\nuse"
 "Biogeochemical\nflows-P"
 "Biogeochemical\nflows-N"
 "Stratospheric\nozone\ndepletion"
 "Land-system\nchange"
 "Biosphere\nIntegrity"

In [ ]:
𝚪ᵦ


10×10 adjoint(::Matrix{Float64}) with eltype Float64:
  1.22241   0.0      -0.0100732   …  -0.0710218    0.122241    0.38386
  0.0       1.22241  -0.0100732      -0.0710218    0.122241    0.38386
  0.325108  0.0       1.06115        -0.0188888    0.0325108   0.4567
 -0.684548  0.0       0.00564097      0.0397722   -0.0684548  -0.214962
  0.220814  0.0       0.0407336      -0.0128293    0.0220814   0.416858
  0.206199  0.0       0.0138327   …  -0.00198018   0.0206199   0.426595
  0.206199  0.0       0.0138327      -0.00198018   0.0206199   0.426595
 -0.134465  0.0       0.00110805      1.00781     -0.0134465  -0.0422246
  0.428032  0.0       0.233494       -0.0248687    1.0428      0.470084
  0.718619  0.0       0.185568       -0.0417518    0.0718619   1.28949

# **2. Background knowledge**

## **2.1 Computational LCA**

First, we explain the basics of technosphere and biosphere exchanges.

In [ ]:
hydro=getAct("electricity production, hydro, run-of-river","DE")


HTML{String}("<table style=\"border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; font-size: 12px;\">\n    <thead>\n        <!-- Top heading row with dark grey background and white bold font -->\n        <tr style=\"background-color: #415a77; font-weight: bold; color: White;\">\n            <td rowspan=\"2\" style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                Activity key: <br><br>  17405\n            </td>\n            <td style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                Activity name\n            </td>\n            <td colspan=\"3\" style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                electricity production, hydro, run-of-river, DE<br>\n            </td>\n        </tr>\n        <tr style=\"background-color: #415a77; font-weight: bold; color: white;\">\n            <td style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                Reference flow\n            </td>\n            <td colspan=\"3\" style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                1.0 kilowatt hour of electricity, high voltage\n            </td>\n        </tr>\n        <td colspan=\"5\" style=\"border: 1px solid #ddd; padding: 7px; background-color: #8d99ae; font-weight: bold; color: White; text-align: center;\">Technosphere Exchanges</td>\n        <!-- Column names row with centered text -->\n        \n    </thead>\n    <tbody>\n        <tr style=\"font-weight: bold;background-color: #8d99ae; padding: 7px; color: White; text-align: center\">\n            <td style=\"border: 1px solid #ddd; background-color: #8d99ae; font-weight: bold; color: White; text-align: center;\">Exchange keys</td>\n\n            <td style=\"border: 1px solid #ddd;\">Reference flow</td>\n            <td style=\"border: 1px solid #ddd;\">Activity</td>\n            <td style=\"border: 1px solid #ddd;\">Amount</td>\n            <td style=\"border: 1px solid #ddd;\">Unit</td>\n        </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">5777</td>\n        <td style=\"border: 1px solid #ddd;\">hydropower plant, run-of-river</td>\n        <td style=\"border: 1px solid #ddd;\">hydropower plant construction, run-of-river - Europe without Switzerland</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">8.07284939292173e-13</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">unit</td>\n    </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">12204</td>\n        <td style=\"border: 1px solid #ddd;\">lubricating oil</td>\n        <td style=\"border: 1px solid #ddd;\">market for lubricating oil - RER</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">7.56e-6</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">kilogram</td>\n    </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">18780</td>\n        <td style=\"border: 1px solid #ddd;\">waste mineral oil</td>\n        <td style=\"border: 1px solid #ddd;\">market for waste mineral oil - Europe without Switzerland</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">-7.56e-6</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">kilogram</td>\n    </tr>\n<table style=\"border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; font-size: 12px;\">\n    <thead>    <tr style=\"font-weight: bold;\">\n        <td colspan=\"5\" style=\"border: 1px solid #ddd; padding: 7px; background-color: #344e41; color: White; text-align: center;\">Biosphere Exchanges</td>\n    </tr>\n<tr style=\"font-weight: bold; background-color: #588157; color: White; text-align: center\">\n\n    <td style=\"border: 1px solid #ddd; padding: 7px; font-weight: bold; color: White; text-align: center;\">Exchange keys</td> \n    <td style=\"border: 1px 

Reference flow               Activity                     Amount       Unit     Location         Key             
──────────────────────────── ──────────────────────────── ──────────── ──────── ──────────────── ────────────────
electricity, high voltage    electricity production, hydro, run-of-river          1.0 kilowatt hour DE               17405           

**Technosphere exchanges**
$$
\boldsymbol{As=f}
\tag{1}
$$


In [31]:
𝐀=Technosphere!().Matrix


23559×23559 SparseMatrixCSC{Float64, Int64} with 291159 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⢸⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡯⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣯⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣽⣷⣺⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣹⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣗⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣇⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⡿⢾⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣯⡯⢹⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣟⢻⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⢹⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⢯⡗⠊⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢯⎦

In [77]:
hydro.key


17405

In [80]:
𝐟 =zeros(𝐀.m)
𝐟[hydro.key]=8.5 # kWh


8.5

In [81]:
𝐬=𝐀\𝐟


23559-element Vector{Float64}:
  1.5961619962937182e-8
  2.009072951671135e-13
  1.207319206381576e-13
  0.0
  6.798853923158172e-6
  1.792522860277065e-11
  1.122022006444139e-9
  7.120530256854345e-8
  0.0
 -0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

**Biosphere exchanges (total inventory)**

$$
\boldsymbol{Bs=BA^{-1}f=g}
\tag{2}
$$


In [87]:
𝐠 =𝐁*𝐬


4709-element Vector{Float64}:
 3.248141519697489e-16
 1.776967885273525e-13
 0.0
 0.00032463596224564773
 1.702117105618006e-5
 0.0
 1.3784605576397657e-11
 0.0
 0.0
 0.0
 ⋮
 6.758261375427807e-13
 0.0
 2.4087665442110117e-8
 0.0
 3.870381735218356e-5
 1.990477946874239e-15
 2.258002612882459e-13
 0.0
 4.431265060806943e-10

In [88]:
𝐁=Biosphere!().Matrix


4709×23559 SparseMatrixCSC{Float64, Int64} with 437621 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⎦

**Impact score calculation**
$$
\boldsymbol{QBs=QBA^{-1}f=h}
\tag{3}
$$

In [89]:
𝐐=Characterisation!().Matrix
𝐡=𝐐*𝐠


10-element Vector{Float64}:
  1.2363529140615868e-14
  9.3239334028255e-13
  2.8627057497295713e-15
  5.176723102445485e-16
  3.4425617703703823e-13
  3.2961996333691245e-13
  7.026791003822627e-14
  4.251897610437602e-18
 -4.338001387522186e-18
  3.6960136184277486e-14

In [92]:
Characterisation!().categories[6]


"Biogeochemical flows-P"

In [85]:
lca(hydro,8.5)


Row,Exchanges,Climate change-Energy imbalance,Climate change-CO2 Concentration,Ocean acidification-Carbonate ion concentration,Atmospheric aerosol loading-Aerosol Optical Depth (AOD),Freshwater use-Global,Biogeochemical flows-P,Biogeochemical flows-N,Stratospheric ozone depletion-Stratospheric O3 concentration,Land-system change-Global,Biosphere Integrity-Change in biosphere integrity
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,"hydropower plant construction, run-of-river",1.22815e-14,9.26216e-13,2.84378e-15,5.17283e-16,3.43363e-13,3.28467e-13,7.01844e-14,4.20628e-18,-4.37449e-18,8.17197e-15
2,market for lubricating oil,3.06242e-17,2.26066e-15,6.95211e-18,3.59448e-19,8.47233e-16,8.09247e-16,8.14846e-17,4.47816e-20,3.65295e-20,2.46845e-17
3,market for waste mineral oil,5.14078e-17,3.917e-15,1.19699e-17,2.95318e-20,4.59514e-17,3.43518e-16,2.06269e-18,8.40502e-22,-3.83121e-23,2.96308e-17
4,"electricity production, hydro, run-of-river",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.8734e-14
5,TOTAL,1.23635e-14,9.32393e-13,2.86271e-15,5.17672e-16,3.44256e-13,3.2962e-13,7.02679e-14,4.2519e-18,-4.338e-18,3.69603e-14


**Absolute environmental sustainability assessment calculation**
$$
\boldsymbol{QBs\oslash\omega =QBA^{-1}f\oslash\omega =h\oslash\omega}
\tag{3}
$$

In [32]:
@load "./tutorial/data_SOS.jld2"


4-element Vector{Symbol}:
 :𝚫𝐗ᴾᵇ
 :𝚫𝐗ᴰ
 :control_var
 :units

In [ ]:
𝚫𝐗ᴾᵇ


10-element Vector{Float64}:
    1.0
   72.0
    0.6880000000000002
    0.07
 4000.0
   11.0
   62.0
   14.0
   25.0
   10.0

In [33]:
α=2e-15 #allocation factor
𝛚=α*𝚫𝐗ᴾᵇ;


In [16]:
⊘ = ./;


In [91]:
𝐝 = 𝐡 ⊘ 𝛚


10-element Vector{Float64}:
  6.181764570307934
  6.474953751962152
  2.0804547599778855
  3.6976593588896316
  0.04303202212962978
 14.982725606223292
  0.5666766938566634
  0.0001518534860870572
 -8.676002775044371e-5
  1.848006809213874

<div style="text-align:center;">
  <img src="./tutorial/PBI.svg" style="height:400px; display:inline-block;">
</div>


## **2.2 Life cycle optimisation models**

### **2.2.1 Rectangularisation of the technosphere matrix**

### **2.2.2 Models**

**Kätelhön, Bardow and Suh, (2016)**

$$
\min_{\text{s.t.}\ s} Z = \boldsymbol{k^T F s} 
\tag{1}
$$

$$
\boldsymbol{As=f} 
\tag{2}
$$


$$
\boldsymbol{s^*\geq 0} 
\tag{3}
$$

$$
\boldsymbol{F s \leq c} 
\tag{4}
$$

<div style="text-align:center;">
  <img src="./tutorial/rectangularisation.svg" style="height:400px; display:inline-block;">
</div>


**Bachmann et al. 2023**

$$
\min_{\text{s.t.}\ s} PF_{CC} = \boldsymbol{q_{CC}^T B s} 
\tag{1}
$$

$$
\boldsymbol{As=f} 
\tag{2}
$$


$$
\boldsymbol{s\geq 0} 
\tag{3}
$$

$$
\boldsymbol{s \leq c} 
\tag{4}
$$
$$
\boldsymbol{qBs \leq PF_{z}} 
\tag{5}
$$

**Letchemberg et al. 2024**

$$
\min_{s, \text{slack}} w^T z
\tag{1}
$$

$$
A s = f + \text{slack}
\tag{2}
$$

$$
s^{\text{low}} \leq s \leq s^{\text{high}}
\tag{3}
$$

$$
0 \leq \text{slack} \leq \text{slack}^{\text{high}}
\tag{4}
$$

$$
z = Q B s
\tag{5}
$$

$$
z \leq z^{\text{high}}
\tag{6}
$$

$$
B s \leq b^{\text{high}}
\tag{7}
$$


**Here, we implement the following model**

$$
\min_{\text{s.t.}\ s} \ \boldsymbol{x  = \Gamma d} 
\tag{1}
$$


$$
\boldsymbol{d = \left(\Lambda s\right)\oslash \omega = \left(Q B s\right)\oslash \omega}
\tag{2}
$$

$$
\boldsymbol{A s = f}
\tag{3}
$$

$$
\boldsymbol{s^j \geq 0}
\tag{4}
$$

$$
\boldsymbol{s^{H_2} \leq c^{H_2}}
\tag{5}
$$
$$
\boldsymbol{s^{el} \leq c^{el}}
\tag{6}
$$

What is the interaction model doing?

<div style="text-align:center;">
  <img src="./tutorial/interactions.svg" style="height:400px; display:inline-block;">
</div>


Okay now let's go to data preparation on activity browser and excel!

# **3.Modelling**


## **3.1 data**

In [ ]:
include("./tutorial/old/matrices.jl");


In [18]:
b = 1:length(processes) # here n refers to the columns of matrix A, which are the processes involved. 
α = 3.5e-12*0.5
ω = (α*𝚫𝐗ᴾᵇ)


10-element Vector{Float64}:
 1.75e-12
 1.26e-10
 1.2040000000000003e-12
 1.225e-13
 7.0e-9
 1.925e-11
 1.085e-10
 2.45e-11
 4.375e-11
 1.75e-11

## **3.2 Optimisation**

In [ ]:
model = Model(CPLEX.Optimizer)
set_silent(model)

# Variables and expressions
@variable(model, 𝐬[1:length(processes)])

@expression(model, 𝐝, Λ*𝐬 ⊘ ω)
# @expression(model, 𝐱, 𝚪ᵦ*𝐝)
@expression(model, 𝐱, 𝐝)
@expression(model, φ, 𝐌*𝐬)

@objective(model, Min, 𝐱);
@constraint(model, 𝐬 ≥ 0)
@constraint(model, A * 𝐬 == f)
@constraint(model, 𝐬 ≤ c)

optimize!(model);
if !is_solved_and_feasible(model)
    error("Solver did not find an optimal solution")
end

# solution_summary(model)
DataFrame(hcat([catnames_ticks,value.(𝐝),value.(𝐱)]...),["Boundaries","𝐝", "𝐱"])


In [ ]:
value.(φ)


In [ ]:
value.(𝐬)


# **Feel free to connect!**

<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:30px;" src="./tutorial/logo_UNSW.svg" width="150">


**Michaël Lejeune**  
*Ph.D. student, UNSW School of Mechanical & Manufacturing Engineering*  
Australian Research Council (ARC) Training Centre for the Global Hydrogen Economy (GlobH2E)  
W: www.globh2e.org.au      
Sustainability in Manufacturing and Life Cycle Engineering Research Group  
W: www.lceresearch.unsw.edu.au  


<img align="left" style="padding-right:30px;" src="./tutorial/frame.svg" width="150"> <img align="left" style="padding-right:30px;" src="./tutorial/frame (1).svg" width="150"><img align="left" style="padding-right:30px;" src="./tutorial/frame (2).svg" width="150"><img align="left" style="padding-right:30px;" src="./tutorial/frame (3).svg" width="150">